In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gym

import numpy as np
from tqdm import tqdm
import os

In [2]:
class SimEnv(gym.Env):
    def __init__(self, b=10, c=0.5, e=3, y0=2, threshold=1, penalty=2, timeout=100):
        '''
        action_space: (y, delta_t)
        observation_space: (x)
        
        Input
        threshold: when x is lower than this threshold, patient die
        penalty: this parameter adds cost to frequent visits
        '''
        super(SimEnv, self).__init__()
        
        self.b = b
        self.c = c
        self.e = e
        self.y0 = y0
        self.threshold = threshold
        self.penalty = penalty
            
        self.action_space = gym.spaces.Box(low=0,high=float("inf"),shape=(2,),dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-float("inf"), high=float("inf"), shape=(1,), dtype=np.float32)
        self.state = None
        self.timeout = timeout
        self.steps_elapsed = 0
        
    def reset(self):
        self.state = np.random.uniform(self.threshold, 15, 1) # numpy array (1,)
        self.steps_elapsed=0
        return(self.state)
        
        
    def step(self, action):
        delta_t=action[1]
        y=action[0]
        next_obs = self.xfunc(x=self.state, delta_t=delta_t, y=action[0], b=self.b,c= self.c, e=self.e, y0=self.y0)
        
        if next_obs > self.threshold:
            reward = delta_t - self.penalty
        elif (-1/self.c*np.log((self.threshold-self.state[0]+self.b-self.e*(2*y*self.y0-y**2))/self.b) < 0):
            reward = -self.penalty 
        else:
            reward = -1/self.c*np.log((self.threshold-self.state[0]+self.b-self.e*(2*y*self.y0-y**2))/self.b) - self.penalty
            
        self.state = next_obs 
        self.steps_elapsed+=1
        
        return(np.array([self.state]).astype(np.float32), reward, next_obs[0] <= self.threshold or self.steps_elapsed > self.timeout , {"x": self.state, "y": action[0], "delta_t": action[1], "dead": next_obs <= self.threshold})
    
    def xfunc(self, x, delta_t, y, b, c, e, y0):
        '''
        Input
        x: last obs/state
        delta_t: time interval btw visits
        y: dosage
        b, c, e, y0: parameters
        Output
        current obs/state
        '''
        return(x-b+b*np.exp(-c*delta_t)+e*(2*y*y0-y**2)) 
    
    def render(self, mode=None):
        pass
    def close(self):
        pass

Actor Critic Policy:
- Two-layer MLP, 64 neurons, activation tanh, output Gaussian mean
- Log standard deviation (same dimension as action)
- Policy: $\mathcal{N}(\mbox{mean}=\mbox{NeuralNet}(s;{W_i, b_i}^L_{i=1}), \mbox{stdev}=e^r)$


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class ActorCriticPolicy(nn.Module):
    def __init__(self, input_dim, output_dim, n_neurons=64, activation=nn.Tanh, distribution = torch.distributions.multivariate_normal.MultivariateNormal):
        '''
        input dim = obs dim
        output dim = action dim
        '''
        super(ActorCriticPolicy, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.n_neurons = n_neurons
        self.distribution = distribution

        # Policy Network
        self.h0 = nn.Linear(input_dim, n_neurons)
        self.h0_act = activation()
        self.h1 = nn.Linear(n_neurons, n_neurons)
        self.h1_act = activation()
        self.output_layer = nn.Linear(n_neurons, output_dim)

        # Value Network
        self.h0v = nn.Linear(input_dim, n_neurons)
        self.h0_actv = activation()
        self.h1v = nn.Linear(n_neurons, n_neurons)
        self.h1_actv = activation()
        self.value_head = nn.Linear(n_neurons, 1)

        self.variance = torch.nn.Parameter(torch.tensor([0.0]*self.output_dim), requires_grad = True)

        #self.mean_activation = nn.Tanh()

    def forward(self, obs, action=None):
        '''
        obs: tensor.size([input_dim])
        '''
        # Policy Forward: obs->action
        x = self.h0(obs)
        x = self.h0_act(x)
        x = self.h1(x)
        x = self.h1_act(x)
        action_mean = self.output_layer(x) #tensor.size([#obs,act_dim])

        action_variance = torch.exp(self.variance)
        action_dist = self.distribution(action_mean, torch.diag_embed(action_variance)) #tensor.size([#obs, obs_dim, obs_dim])

        if action is None:
            action = action_dist.sample() #tensor.size([#obs, act_dim])
        
        logprob = action_dist.log_prob(action) #tensor.size([#obs])
        
        # Value Forward: obs-> state value(V)
        x = self.h0v(obs)
        x = self.h0_actv(x)
        x = self.h1v(x)
        x = self.h1_actv(x)
        value = self.value_head(x)
        value = torch.squeeze(value) #tensor.size([#obs,1]) - > tensor.size([#obs])

        return action, logprob, value # action: tensor.size([#obs, act_dim]), logprob: tensor.size([#obs]), value: tensor.size([#obs])
        # Question: Do we need the entropy of the distribution?
                                           

In [ ]:
class PPO_X():
    def __init__(self, input_dim, output_dim, network=ActorCriticPolicy):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.policy = network(input_dim, output_dim)
        
        self.total_obs = []
        self.total_act = []
        self.total_rew = []
        self.total_ep_rew=[]
        self.total_ep_len=[]
        
    def forward(self, observation, action = None):
        return self.policy.forward(observation, action=action)
    
    def rollout(self, env, timesteps, gamma, lamb): # timesteps << episode length
        '''
        Run policy pi_old in environment for T timesteps
        Compute advantage estimates A_1,...,A_T
        '''
        obs_trajectory = []
        action_trajectory = []
        logprob_trajectory = []
        value_trajectory = []
        reward_trajectory = []
        done_trajectory = []
        
        done = True
        for i in range(timesteps):
            if done:
                obs = torch.unsqueeze(torch.tensor(env.reset()).float(),0) # numpy array shape (obs_dim, ) 
                                                                   # -> tensor.size([obs_dim])
                                                                   # -> tensor.size([1, obs_dim])
                done = False
            
            action, logprob, value = self.forward(observation = obs) # action: tensor.size([1, act_dim]), logprob: tensor.size([1]), value: tensor.size([1])
            
            action = torch.squeeze(action) # tensor([act_dim])
            logprob = torch.squeeze(logprob) # tensor(1)
            value = torch.squeeze(value) # tensor(1)
            
            action_trajectory.append(action)
            logprob_trajectory.append(logprob)
            value_trajectory.append(value)
            obs_trajectory.append(torch.squeeze(obs))
            done_trajectory.append(done)
            
            obs, reward, done, _ = env.step(action.numpy().astype(float)) # step() takes np array shape(act_dim,)
            obs = torch.unsqueeze(torch.tensor(obs).float(),0)
            reward = torch.tensor(reward).float() # tensor.size([])
            #print("reward", reward, "done", done)
            
            reward_trajectory.append(reward)
        # All trajectories are lists of squeezed tensors, done_trajectory is a list of bool
        # List->stacked tensor
        obs_trajectory = torch.stack(obs_trajectory) # tensor.size([#obs, obs_dim])
        action_trajectory = torch.stack(action_trajectory)
        logprob_trajectory = torch.stack(logprob_trajectory)# tensor.size([#obs])
        value_trajectory = torch.stack(value_trajectory)
        reward_trajectory = torch.stack(reward_trajectory)  
        
        done_trajectory = np.asarray(done_trajectory, dtype=np.bool_)
        
        _, __, last_value = self.forward(obs)
        
        adv_trajectory = torch.zeros_like(reward_trajectory).float()
        delta_trajectory = torch.zeros_like(reward_trajectory).float()
        
        for t in reversed(range(timesteps)): #T-1 -> 0
            
            if done_trajectory[t]:
                delta_trajectory[t] = reward_trajectory[t]-value_trajectory[t]
                adv_trajectory[t]=delta_trajectory[t]
            
            else:
                if t == timesteps-1:
                    delta_trajectory[t] = reward_trajectory[t]-value_trajectory[t]
                    adv_trajectory[t]=delta_trajectory[t]
                else:
                    delta_trajectory[t] = reward_trajectory[t]+gamma*value_trajectory[t+1]-value_trajectory[t]
                    adv_trajectory[t]=delta_trajectory[t]+gamma*lamb*adv_trajectory[t+1]
            
#             if t == timesteps-1:
#                 adv_trajectory[t]=delta_trajectory[t]
                
                
#             else:
#                 adv_trajectory[t]=delta_trajectory[t]+gamma*lamb*adv_trajectory[t+1] #recursion
        #print("in rollout:",obs_trajectory.size())
        return(obs_trajectory, action_trajectory, logprob_trajectory, adv_trajectory)
    
    def loss(self, epsilon, obs_traj, action_traj, logprob_traj, adv_traj, current_logprob):
        # ratio
        ratio = torch.exp(current_logprob-logprob_traj) # torch.size([#obs])
        
        # loss
        l1 = ratio*adv_traj # torch.size([#obs])
        l2 = torch.clamp(ratio, 1.-epsilon, 1.+epsilon) # torch.size([#obs])
        # Minimize negative L
        PGloss = -torch.mean(torch.min(l1, l2))# torch.size([#obs])
            
        return PGloss # torch.size([])
    
    def train_step(self, epsilon, obs_traj, action_traj, logprob_traj, adv_traj):
        
        #Question: adv needs normalization?
        
        self.policy.train() # module train mode
        with torch.set_grad_enabled(True):
            action, logprob, value = self.forward(obs_traj, action=action_traj)
            
            PGloss = self.loss(epsilon, obs_traj, action_traj, logprob_traj, adv_traj,logprob)
            
            PGloss.backward()
            
            return(PGloss)
    
    def train(self, env, total_timesteps, timesteps=1024, k_epoch=4, num_batch=4, n_actor=1, 
              epsilon=0.2, gamma = 0.99, lamb=0.95, lr = 0.00027, optimizer = torch.optim.Adam, evaluate=True):
        
        n_updates = total_timesteps//timesteps # number of iterations
        
        self.policy_optimizer=optimizer(self.policy.parameters(), lr=lr)

        for i in tqdm(range(n_updates)):
            actor=0
            obs_trajectory, action_trajectory, logprob_trajectory, adv_trajectory = self.rollout(env, timesteps, gamma, lamb)
            for actor in range(n_actor-1):
                more_obs_traj, more_action_traj, more_logprob_traj, more_adv_traj = self.rollout(env, timesteps, gamma, lamb)
                obs_trajectory = torch.cat((obs_trajectory, more_obs_traj),0)
                action_trajectory = torch.cat((action_trajectory, more_action_traj),0)
                logprob_trajectory = torch.cat((logprob_trajectory, more_logprob_traj),0)
                adv_trajectory = torch.cat((adv_trajectory, more_adv_traj),0)
            
            indices = np.arange(n_actor*timesteps) # [0, 1, ..., n_actor*timesteps-1]
            
            for k in range(k_epoch):
                np.random.shuffle(indices)
                batch_size = n_actor*timesteps//num_batch
                
                if(timesteps%num_batch):
                    batch_size += 1
                
                for b in range(num_batch):
                    #reset gradient
                    self.policy.zero_grad()
                    
                    if b != num_batch-1: # all batches except the last one
                        batch_indices = indices[b*batch_size:(b+1)*batch_size]
                    else:
                        batch_indices = indices[b*batch_size:] # last batch
                
                    batch=[tensor[batch_indices] for tensor in (torch.unsqueeze(obs_trajectory,1), action_trajectory, logprob_trajectory, adv_trajectory)]
                    #print("in train", batch[0])
                    PGloss = self.train_step(epsilon, *batch) 
                    
                    #Question: Do we need to clip gradient?
                    
                    self.policy_optimizer.step()
            if evaluate:
                eval_policy(env)
                    
        def eval_policy(self, env, n_ep=5, timeout=None):
            if timeout==None:
                timeout = env.timeout
                
            for i_episode in range(n_ep):
                obs = []
                act = []
                rew = []
                ep_reward_ls = []
                ep_len_ls=[]
                ep_reward = 0
                ep_len=0
                observation = self.env.reset()
                obs.append(observation)
                for t in range(timeout):
                    action = self.act_sample()
                    observation, reward, done, info = self.env.step(action)
                    #print("reward:", reward)
                    ep_reward += reward
                    ep_len +=1
                    rew.append(reward)
                    act.append(action)
                    obs.append(observation)

                    if done or t == 99:
                        #print("Episode finished after {} timesteps".format(t+1))
                        #print("Survival time: {}".format(ep_reward+env.penalty*(t+1)))
                        break
                        
                ep_reward_ls.append(ep_reward) 
                ep_len_ls.append(ep_len)
                self.total_rew.append(rew)
                self.total_act.append(act)
                self.total_obs.append(obs)

            self.total_ep_rew.append(np.mean(ep_reward))
            self.total_ep_len.append(np.mean(ep_len))
         
    def eval(self, obs):
        with torch.set_grad_enabled(False):
            obs = torch.unsqueeze(torch.tensor(obs).float(),0)
            action, _, __=self.forward(obs)
        return(torch.squeeze(action))
            

In [ ]:
env =SimEnv()

In [ ]:
policy=PPO_X(1,2)

In [ ]:
policy.train(env=env, total_timesteps=10000, timesteps=100) # 100 iterations, 1 actor, 4 epochs, batch size=100/4=25

In [ ]:
torch.transpose(torch.tensor([1.,2.]), 0, 1)

In [ ]:
torch.stack([torch.tensor(1.),torch.tensor(2.)])

In [ ]:
torch.cat((torch.tensor([1.,2.,3.]),torch.tensor([1.,2.,3.])),0)

In [ ]:
ls = []
for i in range(10):
    state=np.random.uniform(1, 15, 4)
    ls.append(torch.squeeze(torch.unsqueeze(torch.tensor(state),0)))
ls

In [ ]:
torch.stack(ls)

In [ ]:
obs2=np.array([[1.,2.,3.,4.,5.], [5.,4.,3.,2.,1.]])
print(np.shape(obs2))
obs2 = torch.tensor(obs2)
obs2.size()

In [ ]:
torch.unsqueeze(obs2, 0)

In [ ]:
obs2

In [ ]:
np.shape(np.array([1,2,3]))

In [ ]:
p = PPO_X(5,4)

In [ ]:
p.forward(obs)

In [ ]:
ac= ActorCriticPolicy(1, 2)

In [ ]:
ac.forward(torch.tensor([[1.],[2.],[3.]]))

In [ ]:
ac(torch.tensor([[1.,2.,3.,4.,5.], [5.,4.,3.,2.,1.]]))

In [ ]:
input_dim=5
n_neurons=64
output_dim=4
obs=torch.tensor([[1.,2.,3.,4.,5.], [5.,4.,3.,2.,1.]])

h0 = nn.Linear(input_dim, n_neurons)
h0_act = nn.Tanh()
h1 = nn.Linear(n_neurons, n_neurons)
h1_act = nn.Tanh()
output_layer = nn.Linear(n_neurons, output_dim)

x = h0(obs)
x = h0_act(x)
x = h1(x)
x = h1_act(x)
action_logit = output_layer(x)

In [ ]:
action_logit.size()

In [ ]:
input_dim=5
n_neurons=64
output_dim=4
obs=torch.tensor([[1.,2.,3.,4.,5.], [5.,4.,3.,2.,1.]])

h0v = nn.Linear(input_dim, n_neurons)
h0_actv = nn.Tanh()
h1v = nn.Linear(n_neurons, n_neurons)
h1_actv = nn.Tanh()
value_head = nn.Linear(n_neurons, 1)

x = h0v(obs)
x = h0_actv(x)
x = h1v(x)
x = h1_actv(x)
value = value_head(x)

In [ ]:
value.size()

In [ ]:
value = torch.squeeze(value)
value.size()

In [ ]:
torch.diag_embed(torch.exp(torch.tensor([0.0]*output_dim)))

In [ ]:
dist = torch.distributions.multivariate_normal.MultivariateNormal(action_logit, torch.diag_embed(torch.exp(torch.tensor([0.0]*output_dim))))
dist

In [ ]:
dist.log_prob(torch.tensor([[1,2,3,4],[2,3,4,5]]))

In [ ]:
dist.log_prob(torch.tensor([[1,2,3,4],[2,3,4,5]])).size()

In [ ]:
torch.tensor([[1,2,3,4]]).size()

In [ ]:
torch.squeeze(torch.tensor([[1,2,3,4]])).size()

In [ ]:
import mujoco_py
env = gym.make('HalfCheetah-v2')
env2 = SimEnv()
P2=PPO2(17,6)
PX=PPO_X(17,6)
P2.train(env=env2) #train policy

In [ ]:
PX.train(env=env, total_timesteps=10000)